# End to End Data Analytics Project (Python + SQL)
## ETL

## Imports

In [1]:
#!pip install kaggle
import kaggle
import pandas as pd
import sqlalchemy as sal

In [2]:
with open("orders.csv", "r", encoding="latin-1") as f:
    content = f.read()

In [3]:
#extract file from zip file
#import zipfile
#zip_ref = zipfile.ZipFile('orders.csv.zip') 3
#zip_ref.extractall() # extract file to dir
#zip_ref.close() # close file

## Read Data & handel null values

In [4]:
df = pd.read_csv('orders.csv', na_values=['Not Available','unknown'])
df.head(20)
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [5]:
#df

In [6]:
df.rename(columns={'Order Id':'order_id', 'City':'city'})
df.columns
df.columns= df.columns.str.lower()
df.columns= df.columns.str.replace(' ','_')
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


#### Derive new columns discount , sale price and profit

In [7]:
df['discount']= df['list_price']*df['discount_percent']*.01

In [8]:
df['sale_price'] = df['list_price']-df['discount']
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0


In [9]:
df['profit'] = df['sale_price'] - df['cost_price']

#### convert order date from object data type to datetime

In [10]:
df.dtypes
df['order_date']=pd.to_datetime(df['order_date'],format="%Y-%m-%d")

In [11]:
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

#### drop cost price list price and discount percent columns

In [12]:
df.drop(columns=['list_price','cost_price','discount_percent'],inplace=True)
df.head(1)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8


#### load the data into sql server using replace option

In [13]:
engine = sal.create_engine(('mssql://Yomna/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER'))
conn=engine.connect()

In [15]:
#df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')

In [16]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')